In [121]:
## Here we will write out the matrix elements for our non-eq setup

from qutip import *
import numpy as np
from scipy import integrate
from helper_code_qutip import * 
import scipy.io

In [122]:
## Firstly, we have to define the integral function C and D

def integral1(i,k,tb,beta,mu,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta,mu,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta,mu,gamma),limit=limit_value)[0]
    return integral

def integral2(i,k,tb,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma),limit=limit_value)[0]
    return integral

def C(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + 0.5*(func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))

    return val

def D(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + integral2(i,k,tb,gamma,eigenergies) + 0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma)+func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))
    return val

In [123]:
NL1 = 1
NL2 = 1
NM = 2

N = NL1 + NL2 + NM
dL1 = 2**NL1
dL2 = 2**NL2
dM = 2**NM
d = 2**N
dims = [2]*N

In [124]:
## Define the Hamiltonian 

w0list = np.linspace(1,1,N)
glist = np.linspace(0.0016,0.0016,N-1)

delta = 1

H_S = create_hamiltonian(w0list,glist,delta,N)

create_sm_list_left = [create_sm(N,i + 1) for i in range(NL1)]
create_sm_list_right = [create_sm(N,NM + NL1 + i + 1) for i in range(NL2)]  # Create list of sigma_- operators

eigenergies,eigstates=H_S.eigenstates()

In [125]:
## Let us compute the thermal density matrix
beta1 = 1  #left baths
beta2 = 1 #right baths

rho_th = (-beta1*H_S).expm()/((-beta1*H_S).expm()).tr() #left thermal density matrix
#print(rho_th)

In [126]:
## Now we set parameters for the  non-eq setup

beta_list_left = np.linspace(beta1,beta1,NL1)
beta_list_right = np.linspace(beta2,beta2,NL2)

mu_list_left = np.linspace(-0.5,-0.5,NL1)
mu_list_right = np.linspace(-0.5,-0.5,NL2)

gamma_list_left = np.linspace(1,1,NL1)  #coupling strengths to left baths
gamma_list_right = np.linspace(1,1,NL2)  #coupling strengths to right baths

tb = 0.01

In [127]:
number = len(eigenergies)

## Now we will write out the matrix elements

A = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        sum = 0
        vi = eigstates[i]
        vk = eigstates[k]
        proj_i = vi*vi.dag()
        proj_k = vk*vk.dag()
        for y in range(number):
            for l in range(NL1):
                proj_y = eigstates[y]*eigstates[y].dag()
                op1 = commutator(proj_k*create_sm_list_left[l]*proj_y,create_sm_list_left[l].dag())*C(k,y,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies)
                sum += vi.dag()*(op1 + op1.dag())*vi

                op2 = commutator(create_sm_list_left[l].dag(),proj_y*create_sm_list_left[l]*proj_k)*D(y,k,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies)
                sum += vi.dag()*(op2 + op2.dag())*vi

            for l in range(NL2):
                proj_y = eigstates[y]*eigstates[y].dag()
                op1 = commutator(proj_k*create_sm_list_right[l]*proj_y,create_sm_list_right[l].dag())*C(k,y,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies)
                sum += vi.dag()*(op1 + op1.dag())*vi

                op2 = commutator(create_sm_list_right[l].dag(),proj_y*create_sm_list_right[l]*proj_k)*D(y,k,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies)
                sum += vi.dag()*(op2 + op2.dag())*vi

        A[i,k] = sum


In [128]:
print(number)  #should be 2^N

16


In [129]:
rho_th_new = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        vi = eigstates[i]
        vk = eigstates[k]

        rho_th_new[i,k] = vi.dag()*rho_th*vk

#print(rho_th_new)

rho_th_diag = [rho_th_new[i,i] for i in range(number)]
print(rho_th_diag)


[np.complex128(0.28671048074431166+0j), np.complex128(0.10547489143421289+0j), np.complex128(0.10527736219305431+0j), np.complex128(0.104802007653906+0j), np.complex128(0.10432879946353751+0j), np.complex128(0.03880204411843679+0j), np.complex128(0.038729377171584346+0j), np.complex128(0.03864492621639909+0j), np.complex128(0.03855450400936416+0j), np.complex128(0.03838042044473365+0j), np.complex128(0.03821890796739024+0j), np.complex128(0.014274474306600173+0j), np.complex128(0.014247741630800462+0j), np.complex128(0.014183409389607018+0j), np.complex128(0.014119367625133617+0j), np.complex128(0.005251285630928183+0j)]


In [130]:
## Now we use the lin algebra solver to solve the equation Ax = b, where b is all zeroes

b = np.zeros((number),dtype=complex)

x = np.linalg.solve(A,b)

print(x)

[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.-0.j -0.-0.j]


In [131]:
print(A)

[[ 5.68155811e-01+0.00000000e+00j -6.37204447e-01+0.00000000e+00j
  -1.08918833e+00+0.00000000e+00j -6.40027480e-01+0.00000000e+00j
  -1.88043603e-01+0.00000000e+00j -6.45083921e-33+0.00000000e+00j
  -1.47855675e-48+0.00000000e+00j -6.46053671e-33+0.00000000e+00j
  -1.40138259e-32+0.00000000e+00j -2.59077727e-32+0.00000000e+00j
  -2.33508020e-48+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [-1.42179530e-01+3.11708125e-19j  1.06346188e+00+2.79588635e-17j
   6.46339427e-18-2.78374617e-17j -4.67046591e-15-1.38981166e-17j
   2.07863033e-17+6.78744721e-18j -9.55806671e-01-1.90819582e-17j
  -5.44594165e-01-4.51028104e-17j  2.95957569e-17-5.57703409e-18j
  -3.20013740e-01-8.67361738e-18j -9.40218013e-02+2.42861287e-17j
  -2.85112612e-17-6.08869478e-18j -7.55234991e-32-5.78545648e-49j
  -2.66377577e-32-2.30607928e-48j -5.59025916e-33+1.46045742e-48j
  -1.1787

In [132]:
## Now we can add one more constraint, that the sum of all the elements of x should be 1, which is basically another row in the matrix A consisting of all 1

#A = np.vstack([A,np.ones((1,number))])

#print(A)

print(np.linalg.matrix_rank(A))

q,r = np.linalg.qr(A.T)

print(r)
diag_r = [r[i][i] for i in range(number)]

print(np.linalg.matrix_rank(A))





15
[[-1.53628035e+00+0.00000000e+00j  4.93674717e-01-1.32141672e-17j
   1.09280869e+00+7.96681822e-17j  4.96816818e-01-6.34889034e-18j
   1.02887148e-01-5.07934683e-18j -1.27898246e-01-1.07106481e-19j
  -1.09621849e-01-4.44911901e-20j -1.32671952e-01-2.76099854e-20j
  -1.47516273e-01-1.00474041e-19j -6.77573370e-02-2.95879935e-20j
  -6.44413625e-02+4.85804292e-20j -5.31383155e-33+1.57607830e-50j
  -4.32002418e-33-3.82406092e-50j -2.67487416e-33+1.13559042e-51j
  -4.69606281e-33+9.92014283e-51j  0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j -1.49293321e+00+0.00000000e+00j
   8.53290445e-02-2.66053493e-17j -1.88885574e-01+1.15988867e-17j
  -1.25852308e-01-1.28287408e-17j  1.10691759e+00-5.16067553e-17j
   6.17955080e-01-1.24917699e-17j -4.38712114e-02-5.41849293e-18j
   3.36549693e-01-1.59388389e-19j  9.06820138e-02+1.44674739e-17j
  -2.13091056e-02-2.55177013e-17j -1.97417635e-01+6.50344739e-18j
  -5.64023537e-02+1.22602388e-18j -7.58996954e-02+4.75451162e-18j
  -3.4

In [133]:
print(diag_r)

[np.complex128(-1.5362803530866331+0j), np.complex128(-1.4929332066905148+0j), np.complex128(-1.2259970638136715+0j), np.complex128(-1.2853964935751205+0j), np.complex128(-1.371264483622374+0j), np.complex128(-1.4566258788164121+0j), np.complex128(-1.5101037357809177+0j), np.complex128(-1.4803349205635714+0j), np.complex128(-1.4550550269977294+0j), np.complex128(-1.4406345436128571+0j), np.complex128(-1.4942701109524872+0j), np.complex128(-1.3001197402368434+0j), np.complex128(-1.7421253674129153+0j), np.complex128(-1.7269851596103611+0j), np.complex128(-0.9373534685480845+0j), np.complex128(-7.992306940441957e-16+0j)]


In [134]:
## Since last element of r is 0, we can remove the last row of  A and add a row of 1's

A_new = A[:-1]
A_new = np.vstack([A_new,np.ones((1,number))])

print(np.linalg.matrix_rank(A_new))
print(A_new.shape)
print(A_new)

16
(16, 16)
[[ 5.68155811e-01+0.00000000e+00j -6.37204447e-01+0.00000000e+00j
  -1.08918833e+00+0.00000000e+00j -6.40027480e-01+0.00000000e+00j
  -1.88043603e-01+0.00000000e+00j -6.45083921e-33+0.00000000e+00j
  -1.47855675e-48+0.00000000e+00j -6.46053671e-33+0.00000000e+00j
  -1.40138259e-32+0.00000000e+00j -2.59077727e-32+0.00000000e+00j
  -2.33508020e-48+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [-1.42179530e-01+3.11708125e-19j  1.06346188e+00+2.79588635e-17j
   6.46339427e-18-2.78374617e-17j -4.67046591e-15-1.38981166e-17j
   2.07863033e-17+6.78744721e-18j -9.55806671e-01-1.90819582e-17j
  -5.44594165e-01-4.51028104e-17j  2.95957569e-17-5.57703409e-18j
  -3.20013740e-01-8.67361738e-18j -9.40218013e-02+2.42861287e-17j
  -2.85112612e-17-6.08869478e-18j -7.55234991e-32-5.78545648e-49j
  -2.66377577e-32-2.30607928e-48j -5.59025916e-33+1.46045742e-4

In [135]:
print(A.shape)

(16, 16)


In [136]:
print(np.linalg.matrix_rank(A))
print(A.shape)

15
(16, 16)


In [137]:
b[-1] = 1  ## Last element of b is 1

x = np.linalg.solve(A_new,b)
print(x)

[0.44807327-9.02759610e-18j 0.09997866+1.59118003e-18j
 0.09979142-8.50746047e-18j 0.09934084+3.13299120e-18j
 0.09889229+5.94563177e-18j 0.02230825+1.06887083e-18j
 0.02226648+4.32511769e-19j 0.02221792+3.91203141e-19j
 0.02216594-6.64677669e-19j 0.02206585+3.70056149e-19j
 0.021973  +3.98522238e-18j 0.00497764+2.84073749e-19j
 0.00496832+1.05013517e-19j 0.00494589+1.18446290e-19j
 0.00492356+3.28735038e-19j 0.00111066+4.45798365e-19j]


In [138]:
##Let us check if this is correct

print(np.dot(A_new,x))
print(np.dot(A[-1],x))

[ 1.74603691e-17-1.46888811e-33j -1.91802450e-17-3.85185989e-34j
 -1.43166440e-18+1.54074396e-33j  2.39541196e-18-7.70371978e-34j
  1.13044278e-17-3.85185989e-34j  1.12159200e-18+3.85185989e-34j
  4.83074129e-18+2.88889492e-34j  2.06653823e-18+0.00000000e+00j
  4.64827816e-18+0.00000000e+00j -7.79500965e-18-1.05926147e-33j
 -5.69466012e-18-7.70371978e-34j  1.62630326e-18-1.32407684e-34j
 -8.67361738e-19+0.00000000e+00j -5.42101086e-19+0.00000000e+00j
 -2.54787511e-18-7.22223729e-35j  1.00000000e+00+9.62964972e-35j]
(-1.951563910473908e-18+1.037236583911637e-18j)


In [139]:
#We have to tranfer the basis of the solution rho matrix to computational basis from the eigenbasis

x_real = [np.real(x[i]) for i in range(number)]

rho = np.diag(x_real)

#set U matrix whose columns are the eigenvectors of the Hamiltonian

U = np.zeros((number,number),dtype=complex)
for i in range(number):
    U[:,i] = eigstates[i].full().flatten()

check = 1

print(U[:,check])
print(eigstates[check])

[ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  5.53594260e-17+0.j  0.00000000e+00+0.j  6.47793817e-22+0.j
 -1.65078136e-16+0.j  5.00000000e-01+0.j  0.00000000e+00+0.j
  1.09923089e-16+0.j  1.26795067e-16+0.j  5.00000000e-01+0.j
  4.84973304e-17+0.j  5.00000000e-01+0.j  5.00000000e-01+0.j
  0.00000000e+00+0.j]
Quantum object: dims=[[2, 2, 2, 2], [1, 1, 1, 1]], shape=(16, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 5.53594260e-17]
 [ 0.00000000e+00]
 [ 6.47793817e-22]
 [-1.65078136e-16]
 [ 5.00000000e-01]
 [ 0.00000000e+00]
 [ 1.09923089e-16]
 [ 1.26795067e-16]
 [ 5.00000000e-01]
 [ 4.84973304e-17]
 [ 5.00000000e-01]
 [ 5.00000000e-01]
 [ 0.00000000e+00]]


In [140]:
rho_comp = np.dot(np.conjugate(U.T),np.dot(rho,U))

print(rho_comp)
print(np.trace(rho_comp))

[[ 1.11066259e-03+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  9.26625359e-03+0.j -5.65334107e-03+0.j
   4.30448679e-03+0.j -2.32953836e-03+0.j -2.35413633e-18+0.j
   2.68117882e-18+0.j  3.91205492e-18+0.j -3.11022744e-18+0.j
  -2.00996817e-18+0.j -1.20658815e-18+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j -5.65334107e-03+0.j  1.23099854e-02+0.j
  -5.64474818e-03+0.j  3.04373180e-03+0.j  5.30412793e-19+0.j
  -1.25014332e-18+0.j -2.40637814e-18+0.j -2.98665047e-19+0.j
  -2.49039457e-19+0.j  4.40139292e-19+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 

In [141]:
def L2_red(rho,eigenergies,eigstates,beta_list_left,beta_list_right,mu_list_left,mu_list_right,gamma_list_left,gamma_list_right,tb):
    sum = 0
    rho = Qobj(rho)
    rho.dims = [dims,dims]
    for i in range(number):
        for k in range(number):
            vi = eigstates[i]
            vk = eigstates[k]

            proj_i = vi*vi.dag()
            proj_k = vk*vk.dag()

            for l in range(NL1):
                op = commutator(rho*proj_i*create_sm_list_left[l]*proj_k,create_sm_list_left[l].dag())*C(i,k,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies) + commutator(create_sm_list_left[l].dag(),proj_i*create_sm_list_left[l]*proj_k*rho)*D(i,k,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies)
                sum += op
                sum += op.dag()

            for l in range(NL2):
                op = commutator(rho*proj_i*create_sm_list_right[l]*proj_k,create_sm_list_right[l].dag())*C(i,k,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies) + commutator(create_sm_list_right[l].dag(),proj_i*create_sm_list_right[l]*proj_k*rho)*D(i,k,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies)
                sum += op
                sum += op.dag()
    data = sum.full()
    sum = np.array(data,dtype=complex)
    return sum

In [142]:
L2_redfield = L2_red(rho_comp,eigenergies,eigstates,beta_list_left,beta_list_right,mu_list_left,mu_list_right,gamma_list_left,gamma_list_right,tb)

In [143]:
print(L2_redfield)

[[-6.12380114e-03+0.00000000e+00j -6.11409480e-07+1.56986312e-06j
  -1.24128069e-03-5.15054303e-06j -8.07142856e-09+2.09263080e-08j
  -3.42378297e-09-6.38503458e-09j  2.35263918e-11+4.36298964e-11j
   1.80458610e-13-4.15075592e-17j  4.50565129e-24-1.31970488e-23j
  -3.66090696e-11-5.01951294e-11j  8.07910934e-16+4.54041966e-17j
   4.25337460e-18-9.69347246e-19j -6.38025259e-24+2.46964719e-24j
  -5.96020773e-22-4.10691195e-21j  6.55260773e-26+3.25986545e-25j
   1.98548072e-24-4.70571135e-24j  0.00000000e+00+0.00000000e+00j]
 [-6.11409480e-07-1.56986312e-06j  5.45930286e-03+0.00000000e+00j
  -1.05022864e-02-9.68348072e-03j  6.69631186e-03+1.03082212e-05j
  -4.78184495e-03-3.98933802e-03j  1.39879073e-08+2.08815552e-08j
   3.16107805e-09-1.20745854e-08j -1.06518252e-14+3.97705182e-14j
   3.22573867e-07-8.30244945e-07j -1.74855830e-09-3.24275130e-09j
  -7.79203147e-13-2.99986396e-12j -2.18734573e-20+3.68439746e-21j
  -3.61852475e-18+2.16746668e-15j  1.91804395e-20+1.76432328e-22j
   9.8570

In [144]:
## As we can see, all the checks are completely satisfied. Thus we can arrange the values of x (real values) in a diagonal density matrix and save it in a .mat file
#We will also pass in L2_redfield operator

rho_th_check = np.zeros((number,number),dtype=complex)
for i in range(number):
    for k in range(number):
        rho_th_check[i,k] = rho_th[i,k]

print(rho_th_check)

data_dict = {"dm_ness":rho_comp, "dm_th":rho_th_new,"L2_red":L2_redfield, "dm_th_check":rho_th_check}

scipy.io.savemat(F'../matlab/ness_data_NL1 = {NL1}_NL2 = {NL2}_NM = {NM}_6.mat',data_dict)


[[5.25128563e-03+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j]
 [0.00000000e+00+0.j 1.42289418e-02+0.j 4.54597617e-05+0.j
  0.00000000e+00+0.j 7.26967645e-08+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 7.75845012e-11+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j]
 [0.00000000e+00+0.j 4.54597617e-05+0.j 1.41835547e-02+0.j
  0.00000000e+00+0.j 4.53871426e-05+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 7.26967645e-08+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  0.00000000e+00+0.j]
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.0000000